In [ ]:
import json
import pandas as pd
import firebase_admin
from datetime import date
from firebase_admin import credentials
from firebase_admin import firestore

In [ ]:

df = pd.read_csv('C:\\Users\\fmabd\\Documents\\GitHub\\Smart-Customer-Care\\Text Normalization\\Tweets.csv')

In [ ]:
df

In [ ]:
df.dropna(inplace= True)

In [ ]:
df.info()

In [ ]:
CredNew = credentials.Certificate('C:\\Users\\fmabd\\Documents\\GitHub\\Smart-Customer-Care\\Database\\smartcc-960f7-firebase-adminsdk-ijo0i-27aa89d85e.json')
new=firebase_admin.initialize_app(CredNew,name="new")

db = firestore.client(new)

In [ ]:
csv= json.loads(json.dumps(df[0:3000].to_dict()))

In [ ]:
csv2= json.loads(json.dumps(df[3000:].to_dict()))

In [ ]:


# datetime object containing current date and time
today = date.today()
today= str(now)


today1 = date.today()
today1= str(1)

In [5]:


today = date.today()

In [7]:
print(today)

2022-04-06


In [ ]:
tweets= db.collection(u'test').document(now).set(csv)
tweets1= db.collection(u'test').document(now1).set(csv2)

# will use these commented codes in upcoming phases # 

In [ ]:
'''
DataOld=[]
ids=[]
all_users_ref_2 = db.collection(u'test').stream()
for users in all_users_ref_2:
    #print(u'{} => {}'.format(users.id, users.to_dict()))
    DataOld.append(users.to_dict())
    ids.append(users.id)
'''

In [ ]:
'''
dataNew=[]
idsNew=[]
all_users_ref_2 = db.collection(u'test').stream()
for users in all_users_ref_2:
    #print(u'{} => {}'.format(users.id, users.to_dict()))
    dataNew.append(users.to_dict())
    idsNew.append(users.id)
'''

In [ ]:
#compare between new data and old one
'''
for i in range(len(DataOld)):
    if DataOld[i] not in dataNew:
        db.collection(u'test').document(ids[i]).set(DataOld[i])
    else:
        print("data already exists")
    '''

In [ ]:
#read old data from old firebase account
'''
# Use a service account
CredOld = credentials.Certificate('C:\\Users\\fmabd\\Documents\\GitHub\\Smart-Customer-Care\\Database\\smartcc-26e57-85547b8418c0.json')
old=firebase_admin.initialize_app(CredOld,name="old")

db = firestore.client(old)
'''

In [ ]:
# read full collection in the new account
'''

database_2 = firestore.client()
all_users_ref_2 = database_2.collection(u'test').stream()
for users in all_users_ref_2:
  print(u'{} => {}'.format(users.id, users.to_dict()))

'''